In [0]:
import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import svm
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
url='https://raw.githubusercontent.com/scarface961/Bank-Marketting/master/bank.csv'
data = pd.read_csv(url,delimiter=';')
#data1 = pd.read_csv("D:/Data Mining and Concepts Learning/Project/bank/bank.csv")

In [29]:
data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [0]:
data=pd.get_dummies(data, columns=["job","marital","education","default","housing","loan","contact","month","poutcome"])

labelencoder = LabelEncoder()
data['y'] = labelencoder.fit_transform(data['y'])

In [31]:
data

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,1787,19,79,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,33,4789,11,220,1,339,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
4,59,0,5,226,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,-333,30,329,5,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4517,57,-3313,9,153,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4518,57,295,19,151,11,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4519,28,1137,6,129,4,211,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
def epoch (history):
  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [0]:
def percentage_split_NN(percentage_split,X,Y):
  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)

  model= create_NN_model()
  history = model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs=150, batch_size=100)
  pred = model.predict_classes(x_test)
  tn,fp,fn,tp = confusion_matrix(y_test,pred).ravel()
  print("Percentage Split")
  Confusion_matrix(tn,fp,fn,tp)
  #epoch(history)
  

In [0]:
def crossvalidate_NN(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    model= create_NN_model()
    history = model.fit(X[train], Y[train],validation_data=(X[test],Y[test]),epochs=150, batch_size=100)
    pred = model.predict_classes(X[test])
    #evaluate the model
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],pred).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1

    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
  print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    
  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def percentage_split_SVM(percentage_split,X,Y):

  x_train,x_test,y_train,y_test = train_test_split(X,Y,stratify=Y,test_size = percentage_split,random_state=0)
  clf = svm.SVC(kernel="linear")
  clf.fit(x_train, y_train)
  predict_svm = clf.predict(x_test)
  Confusion_matrix(y_test,predict_svm)


In [0]:
def crossvalidate_SVM(folds,X,Y):
  kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
  cvscores = []
  tn=0
  fp=0
  fn=0
  tp=0
  for train, test in kfold.split(X, Y):
    clf = svm.SVC(kernel="linear")
    clf.fit(X[train], Y[train])
    predict_svm = clf.predict(X[test])
    tn1, fp1, fn1, tp1 = confusion_matrix(Y[test],predict_svm).ravel()
    tn = tn+tn1
    fp = fp+fp1
    fn = fn+fn1
    tp = tp+tp1


    print("Acc: "+repr(accuracy_score(Y[test],predict_svm)))

  tn=tn/folds
  fp=fp/folds
  fn=fn/folds
  tp=tp/folds
  print("Cross-validation")
  Confusion_matrix(tn,fp,fn,tp)


In [0]:
def Confusion_matrix(tn, fp, fn, tp):
  #tn, fp, fn, tp = confusion_matrix(y_test,pred).ravel()
  print("Confusion Matrix")
  print("   0     1   << Classified as   ")
  # print("   %.2f    %.2f" %(tp,fn))
  # print("   %.2f    %.2f" %(fp,tn))
  print("   "+repr(int(tp))+"    "+repr(int(fn)))
  print("   "+repr(int(fp))+"    "+repr(int(tn)))
  accuracy = float((tp+tn)/(tp+tn+fp+fn))
  specificity = float(tn / (tn+fp))
  sensitivity = float(tp / (tp+fn))
  #print(classification_report(y_test,predictions))
  print("Accuracy: "+repr(accuracy))
  print("Sensitivity: "+repr(sensitivity))
  print("Specificity: "+repr(specificity))

In [0]:
def create_NN_model():
  #Keras Multi Layer
  model = Sequential()
  model.add(Dense(5,input_dim=51,activation='relu'))
  model.add(Dense(5, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
  #sgd = optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
  #a = optimizers.adam(lr=0.007)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [0]:
# Splitting the data labels and dataset
Y = data.y.values
X= data.drop('y',axis = 1).values
# X = data.iloc[:,0:53]
# Y = data.iloc[:,52:53]
# print(X)
# #Normalize
# from sklearn import preprocessing
# X = preprocessing.scale(X)
# print(Y)





In [55]:
percentage_split_NN(0.25,X,Y)

Train on 3390 samples, validate on 1131 samples
Epoch 1/150
3390/3390 [==============================] - 1s 422us/step - loss: 11.5059 - acc: 0.1322 - val_loss: 9.3964 - val_acc: 0.1839
Epoch 2/150
3390/3390 [==============================] - 0s 21us/step - loss: 4.9956 - acc: 0.4398 - val_loss: 2.4374 - val_acc: 0.7542
Epoch 3/150
3390/3390 [==============================] - 0s 19us/step - loss: 2.5137 - acc: 0.7569 - val_loss: 2.1218 - val_acc: 0.7666
Epoch 4/150
3390/3390 [==============================] - 0s 21us/step - loss: 1.9287 - acc: 0.7676 - val_loss: 1.5281 - val_acc: 0.7560
Epoch 5/150
3390/3390 [==============================] - 0s 18us/step - loss: 1.5334 - acc: 0.8091 - val_loss: 1.2633 - val_acc: 0.8161
Epoch 6/150
3390/3390 [==============================] - 0s 18us/step - loss: 1.3687 - acc: 0.8274 - val_loss: 1.1203 - val_acc: 0.8355
Epoch 7/150
3390/3390 [==============================] - 0s 21us/step - loss: 1.1861 - acc: 0.8475 - val_loss: 1.0062 - val_acc: 0.850

In [41]:
crossvalidate_NN(10,X,Y)

Train on 4068 samples, validate on 453 samples
Epoch 1/150
4068/4068 [==============================] - 1s 125us/step - loss: 2.4963 - acc: 0.6382 - val_loss: 2.0593 - val_acc: 0.6203
Epoch 2/150
4068/4068 [==============================] - 0s 16us/step - loss: 1.5363 - acc: 0.6640 - val_loss: 1.2152 - val_acc: 0.6424
Epoch 3/150
4068/4068 [==============================] - 0s 16us/step - loss: 0.9050 - acc: 0.6792 - val_loss: 0.6747 - val_acc: 0.6600
Epoch 4/150
4068/4068 [==============================] - 0s 14us/step - loss: 0.5400 - acc: 0.8324 - val_loss: 0.4829 - val_acc: 0.8830
Epoch 5/150
4068/4068 [==============================] - 0s 16us/step - loss: 0.4438 - acc: 0.8850 - val_loss: 0.4335 - val_acc: 0.8830
Epoch 6/150
4068/4068 [==============================] - 0s 15us/step - loss: 0.4117 - acc: 0.8850 - val_loss: 0.4113 - val_acc: 0.8830
Epoch 7/150
4068/4068 [==============================] - 0s 13us/step - loss: 0.3914 - acc: 0.8850 - val_loss: 0.3894 - val_acc: 0.8830


In [0]:
# clf = svm.SVC(kernel="linear")
# clf.fit(x_train, y_train)
# predict_svm = clf.predict(x_test)

In [0]:
# Confusion_matrix(y_test,predict_svm)

Confusion Matrix
   0     1   << Classified as   
   29    101
   35    966
Accuracy: 0.8797524314765695
Sensitivity: 0.2230769230769231
Specificity: 0.965034965034965


In [0]:
crossvalidate_SVM(10,X,Y)

Acc: 0.8741721854304636
Acc: 0.8761061946902655
Acc: 0.8849557522123894
Acc: 0.8938053097345132
Acc: 0.8915929203539823
Acc: 0.8738938053097345
Acc: 0.8761061946902655
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Acc: 0.8938053097345132
Cross-validation
Confusion Matrix
   0     1   << Classified as   
   15    37
   14    385
Accuracy: 0.8815207780725022
Sensitivity: 0.28982725527831094
Specificity: 0.9627500000000001
